### Package version
- tensorflow==2.2.0rc3
- pandas==1.0.3

# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


In [18]:
import tensorflow
tensorflow.__version__

'2.2.0-rc3'

## Get data

### Import CIFAR10 data

### Get train and test data
Use these variable names

X_train, X_test, y_train, y_test

In [0]:
from tensorflow.keras.datasets import cifar10
(x_train,y_train), (x_test,y_test) = cifar10.load_data()

### Print the shape of X_train and y_train

In [20]:
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


### Rehshape y_train and y_test
reshape with y_train.shape[0] and y_test.shape[0]

In [0]:
y_train = y_train.reshape(y_train.shape[0])
y_test = y_test.reshape(y_test.shape[0])

### Create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
x_train_lt5 = x_train[y_train<5]
y_train_lt5 = y_train[y_train<5]
x_test_lt5 = x_test[y_test<5]
y_test_lt5 = y_test[y_test<5]

x_train_gt5 = x_train[y_train>=5]
y_train_gt5 = y_train[y_train>=5]
x_test_gt5 = x_test[y_test>=5]
y_test_gt5 = y_test[y_test>=5]

## Use One-hot encoding to divide y_train and y_test into required no of output classes
Do it for both datasets

In [0]:
import pandas as pd
y_train_lt5 = pd.get_dummies(y_train_lt5)
y_test_lt5 = pd.get_dummies(y_test_lt5)

y_train_gt5 = pd.get_dummies(y_train_gt5)
y_test_gt5 = pd.get_dummies(y_test_gt5)

### Print data variables for dataset having classes from 0 to 4
Data variables here are referring to X-train, X_test, y_train, y_test of that dataset

In [24]:
print(x_train_lt5.shape)
print(y_train_lt5.shape)
print(x_test_lt5.shape)
print(y_test_lt5.shape)

(25000, 32, 32, 3)
(25000, 5)
(5000, 32, 32, 3)
(5000, 5)


### Print data variables for dataset having classes from 5 to 9
Data variables here are referring to X-train, X_test, y_train, y_test of that dataset

In [25]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(25000, 32, 32, 3)
(25000, 5)
(5000, 32, 32, 3)
(5000, 5)


## Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

### Import required libraries

### Initialize a model and add the required layers

In [0]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,BatchNormalization,MaxPooling2D,Dropout, Convolution2D, Dense

model = Sequential()
model.add(BatchNormalization(input_shape = (32,32,3)))
model.add(Convolution2D(32,(3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))
model.add(Convolution2D(64,(3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))

model.add(Flatten())
#model.add(BatchNormalization())
model.add(Dense(128,activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(5,activation = 'softmax'))

### Summarize your model

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 6, 6, 64)         

### Compile and fit the model

In [28]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(lr = 0.001)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy',metrics = ['accuracy'])
model.fit(x_train_lt5, y_train_lt5, validation_data=(x_test_lt5,y_test_lt5), epochs = 5, batch_size= 32) 

Epoch 1/5
782/782 [==============================] - 38s 49ms/step - loss: 0.9997 - accuracy: 0.5994 - val_loss: 0.7840 - val_accuracy: 0.7140
Epoch 2/5
782/782 [==============================] - 38s 49ms/step - loss: 0.7775 - accuracy: 0.7026 - val_loss: 0.6613 - val_accuracy: 0.7528
Epoch 3/5
782/782 [==============================] - 39s 49ms/step - loss: 0.6827 - accuracy: 0.7416 - val_loss: 0.6306 - val_accuracy: 0.7716
Epoch 4/5
782/782 [==============================] - 39s 50ms/step - loss: 0.6210 - accuracy: 0.7676 - val_loss: 0.5918 - val_accuracy: 0.7826
Epoch 5/5
782/782 [==============================] - 39s 49ms/step - loss: 0.5774 - accuracy: 0.7818 - val_loss: 0.5765 - val_accuracy: 0.7832


### Evaluate your model

In [29]:
model.evaluate(x_test_lt5,y_test_lt5)

157/157 [==============================] - 2s 12ms/step - loss: 0.5765 - accuracy: 0.7832


[0.5764948725700378, 0.7832000255584717]

## In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [30]:
for layers in model.layers:
    print(layers.name)
    if('dense' not in layers.name):
        layers.trainable = False

batch_normalization_1
conv2d_2
max_pooling2d_2
dropout_3
conv2d_3
max_pooling2d_3
dropout_4
flatten_1
dense_2
dropout_5
dense_3


### Print in colors

In [31]:
#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

batch_normalization_1
False
conv2d_2
False
max_pooling2d_2
False
dropout_3
False
conv2d_3
False
max_pooling2d_3
False
dropout_4
False
flatten_1
False
dense_2
True
dropout_5
False
dense_3
True


## Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [32]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(lr = 0.001)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train_gt5, y_train_gt5, validation_data = (x_test_gt5, y_test_gt5), epochs = 5, batch_size = 32)

Epoch 1/5
782/782 [==============================] - 15s 20ms/step - loss: 0.8821 - accuracy: 0.6744 - val_loss: 0.5109 - val_accuracy: 0.8224
Epoch 2/5
782/782 [==============================] - 15s 20ms/step - loss: 0.5457 - accuracy: 0.8040 - val_loss: 0.4463 - val_accuracy: 0.8466
Epoch 3/5
782/782 [==============================] - 15s 20ms/step - loss: 0.4812 - accuracy: 0.8268 - val_loss: 0.4201 - val_accuracy: 0.8532
Epoch 4/5
782/782 [==============================] - 15s 20ms/step - loss: 0.4398 - accuracy: 0.8415 - val_loss: 0.3964 - val_accuracy: 0.8602
Epoch 5/5
782/782 [==============================] - 16s 20ms/step - loss: 0.4084 - accuracy: 0.8515 - val_loss: 0.3891 - val_accuracy: 0.8602


In [33]:
model.evaluate(x_test_gt5, y_test_gt5)

157/157 [==============================] - 2s 12ms/step - loss: 0.3891 - accuracy: 0.8602


[0.38910984992980957, 0.8601999878883362]